In [4]:
!pip install -i https://pypi.tuna.tsinghua.edu.cn/simple tensorflow==2.2.3

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     |████████████████████████████████| 175.7 MB 1.1 MB/s eta 0:00:014
     |████████████████████████████████| 42 kB 4.6 MB/s  eta 0:00:01
     |████████████████████████████████| 132 kB 12.0 MB/s eta 0:00:01
     |████████████████████████████████| 15.1 MB 5.7 MB/s eta 0:00:011
     |████████████████████████████████| 1.0 MB 12.2 MB/s eta 0:00:01
     |████████████████████████████████| 57 kB 10.2 MB/s eta 0:00:01
     |████████████████████████████████| 454 kB 6.1 MB/s eta 0:00:01
     |████████████████████████████████| 3.0 MB 16.8 MB/s eta 0:00:01
     |████████████████████████████████| 3.9 MB 9.0 MB/s eta 0:00:01
     |████████████████████████████████| 3.0 MB 11.4 MB/s eta 0:00:01
     |████████████████████████████████| 65 kB 5.9 MB/s  eta 0:00:01
     |████████████████████████████████| 781 kB 6.0 MB/s eta 0:00:01
     |████████████████████████████████| 97 kB 7.0 MB/s  eta 0:00:01
     |████████████████████████████████| 288 kB

     |████████████████████████████████| 77 kB 4.9 MB/s eta 0:00:011
Using legacy 'setup.py install' for wrapt, since package 'wheel' is not installed.
Using legacy 'setup.py install' for termcolor, since package 'wheel' is not installed.
    Running setup.py install for wrapt ... done
    Running setup.py install for termcolor ... done
You should consider upgrading via the '/Users/wizhang/.pyenv/versions/3.8.2/bin/python -m pip install --upgrade pip' command.


In [7]:
import numpy as np
import tensorflow as tf

print(tf.__version__)

2.2.3


In [8]:
users = ['Rayman', 'Daniell', 'Wikcy', 'Chris']
movies = ['Star Wars', 'The Dark Knight', 'Shrek', 'The Incredibles', 'Bleu', 'Memoto']
features = ['Action', 'Sci-Fi', 'Comedy', 'Cartoon', 'Drama']

num_users = len(users)
num_movies = len(movies)
num_features = len(features)
num_recommdations = 2

In [9]:
# user reates movies
users_movies = tf.constant(
[
        [4, 6, 8, 0, 0, 0],
        [0, 0, 10, 0, 8, 3],
        [0, 6, 0, 0, 3, 7],
        [10, 9, 0, 5, 0, 2]
], dtype = tf.float32
)

# movie has features
movies_features = tf.constant([
        [1, 1, 0, 0, 1],
        [1, 1, 0, 0, 0],
        [0, 0, 1, 1, 0],
        [1, 0, 1, 1, 0],
        [0, 0, 0, 0, 1],
        [1, 0, 0, 0, 1]
], dtype = tf.float32)


In [10]:
# Mattrix: the user rated features
users_features = tf.matmul(users_movies, movies_features)
users_features

<tf.Tensor: shape=(4, 5), dtype=float32, numpy=
array([[10., 10.,  8.,  8.,  4.],
       [ 3.,  0., 10., 10., 11.],
       [13.,  6.,  0.,  0., 10.],
       [26., 19.,  5.,  5., 12.]], dtype=float32)>

In [11]:
# scale up
users_features = users_features / tf.reduce_sum(users_features, axis=1, keepdims=True)
users_features

<tf.Tensor: shape=(4, 5), dtype=float32, numpy=
array([[0.25      , 0.25      , 0.2       , 0.2       , 0.1       ],
       [0.0882353 , 0.        , 0.29411766, 0.29411766, 0.32352942],
       [0.44827586, 0.20689656, 0.        , 0.        , 0.3448276 ],
       [0.3880597 , 0.2835821 , 0.07462686, 0.07462686, 0.17910448]],
      dtype=float32)>

In [12]:
# Rank the user rated features that each of the users like the most so far
# nn = neural network
top_users_features = tf.nn.top_k(users_features, num_features)[1]
top_users_features # indexes of which movie which user like the most

<tf.Tensor: shape=(4, 5), dtype=int32, numpy=
array([[0, 1, 2, 3, 4],
       [4, 2, 3, 0, 1],
       [0, 4, 1, 2, 3],
       [0, 1, 4, 2, 3]], dtype=int32)>

In [15]:
for i in range(num_users):
    feature_names = [features[int(index)] for index in top_users_features[i]]
    print('{}: {}'.format(users[i], feature_names))


Rayman: ['Action', 'Sci-Fi', 'Comedy', 'Cartoon', 'Drama']
Daniell: ['Drama', 'Comedy', 'Cartoon', 'Action', 'Sci-Fi']
Wikcy: ['Action', 'Drama', 'Sci-Fi', 'Comedy', 'Cartoon']
Chris: ['Action', 'Sci-Fi', 'Drama', 'Comedy', 'Cartoon']


In [16]:
users_ratings = tf.matmul(users_features, tf.transpose(movies_features))
users_ratings

<tf.Tensor: shape=(4, 6), dtype=float32, numpy=
array([[0.6       , 0.5       , 0.4       , 0.65      , 0.1       ,
        0.35      ],
       [0.4117647 , 0.0882353 , 0.5882353 , 0.67647064, 0.32352942,
        0.4117647 ],
       [1.        , 0.6551724 , 0.        , 0.44827586, 0.3448276 ,
        0.79310346],
       [0.8507463 , 0.6716418 , 0.14925373, 0.53731346, 0.17910448,
        0.5671642 ]], dtype=float32)>

In [20]:
users_ratings_new = tf.where(
    tf.equal(users_movies, tf.zeros_like(users_movies)),
    users_ratings,
    tf.zeros_like(tf.cast(users_movies, tf.float32))
)
users_ratings_new

<tf.Tensor: shape=(4, 6), dtype=float32, numpy=
array([[0.        , 0.        , 0.        , 0.65      , 0.1       ,
        0.35      ],
       [0.4117647 , 0.0882353 , 0.        , 0.67647064, 0.        ,
        0.        ],
       [1.        , 0.        , 0.        , 0.44827586, 0.        ,
        0.        ],
       [0.        , 0.        , 0.14925373, 0.        , 0.17910448,
        0.        ]], dtype=float32)>

In [21]:
top_movies = tf.nn.top_k(users_ratings_new, num_recommdations)[1]
top_movies

<tf.Tensor: shape=(4, 2), dtype=int32, numpy=
array([[3, 5],
       [3, 0],
       [0, 3],
       [4, 2]], dtype=int32)>

In [25]:
for i in range(num_users):
    movie_names = [movies[index] for index in top_movies[i]]
    print('{}: {}'.format(users[i], movie_names))

Rayman: ['The Incredibles', 'Memoto']
Daniell: ['The Incredibles', 'Star Wars']
Wikcy: ['Star Wars', 'The Incredibles']
Chris: ['Bleu', 'Shrek']
